In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import keras
from keras import models
from keras import layers
from csv import DictReader
import csv
import random

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

genderclassmodel.csv
gendermodel.csv
gendermodel.py
myfirstforest.py
test.csv
train.csv



In [4]:
people = list(DictReader(open('../input/train.csv')))
people

[OrderedDict([('PassengerId', '1'),
              ('Survived', '0'),
              ('Pclass', '3'),
              ('Name', 'Braund, Mr. Owen Harris'),
              ('Sex', 'male'),
              ('Age', '22'),
              ('SibSp', '1'),
              ('Parch', '0'),
              ('Ticket', 'A/5 21171'),
              ('Fare', '7.25'),
              ('Cabin', ''),
              ('Embarked', 'S')]),
 OrderedDict([('PassengerId', '2'),
              ('Survived', '1'),
              ('Pclass', '1'),
              ('Name', 'Cumings, Mrs. John Bradley (Florence Briggs Thayer)'),
              ('Sex', 'female'),
              ('Age', '38'),
              ('SibSp', '1'),
              ('Parch', '0'),
              ('Ticket', 'PC 17599'),
              ('Fare', '71.2833'),
              ('Cabin', 'C85'),
              ('Embarked', 'C')]),
 OrderedDict([('PassengerId', '3'),
              ('Survived', '1'),
              ('Pclass', '3'),
              ('Name', 'Heikkinen, Miss. Laina'),
   

In [5]:
def letter_for_cabin(cabin):
    return cabin[0] if len(cabin) else ''

cabin_letters = list(set([letter_for_cabin(p['Cabin']) for p in people]))
print(cabin_letters)

['', 'B', 'A', 'D', 'T', 'C', 'E', 'G', 'F']


In [6]:
def onehot(idx, maximum):
    v = [0] * maximum
    v[idx] = 1
    return v

def avg_for_field(field):
    vals = [float(p[field]) for p in people if p[field]]
    return sum(vals) / len(vals)

avg_age = avg_for_field('Age')
avg_fare = avg_for_field('Fare')

def vectorize(person):
    pclass = int(person['Pclass'])
    salutations = [1 if sal in person['Name'] else 0 for sal in ['Mr.', 'Miss.', 'Mrs.', 'Master.']]
    embarked = 'CQS'.index(person['Embarked'])
    age = float(person['Age']) if person['Age'] else avg_age
    
    sibs = 0
    parents = 0
    spouses = 0
    children = 0
    if age > 18:
        children = int(person['Parch'])
        spouses = int(person['SibSp'])
    else:
        parents = int(person['Parch'])
        sibs = int(person['SibSp'])
    
    sex = -1 if person['Sex'] == 'male' else 1
    fare = float(person['Fare']) if person['Fare'] else avg_fare
    
    cabin = cabin_letters.index(letter_for_cabin(person['Cabin']))
    
    return np.array([pclass, sex, fare, age] + salutations + onehot(embarked, 3) + [sibs, parents, spouses, children] + onehot(cabin, len(cabin_letters)))

def make_vecs_and_labels(people):
    xs = np.stack([vectorize(p) for p in people])
    ys = np.stack([int(p['Survived']) for p in people])
    return xs, ys

test_split = 0.8
random.shuffle(people)

# train = people[:int(len(people) * test_split)]
# test = people[int(len(people) * test_split):]
input_dim = len(vectorize(people[0]))

In [7]:
x, y = make_vecs_and_labels(people)

def create_model(n_layers=3, layer_size=256, activation='relu', dropout=0):
    model = models.Sequential()
    prev_dim = input_dim
    for i in range(n_layers):
        model.add(layers.Dense(layer_size, input_dim=prev_dim))
        prev_dim = layer_size
        model.add(layers.Activation(activation))
        if dropout > 0:
            model.add(layers.Dropout(dropout))
    model.add(layers.Dense(2))
    model.add(layers.Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [8]:
options = {
    'n_layers': [1, 2, 3],
    'layer_size': [128, 256, 512],
    'activation': ['relu', 'sigmoid'],
    'dropout': [0, 0.1]
}
def all_configs(options):
    if len(options) == 0:
        return [[]]
    else:
        child_configs = all_configs(options[1:])
        configs = []
        key = options[0][0]
        for value in options[0][1]:
            for child in child_configs:
                configs.append([(key, value)] + child)
        return configs
configs = all_configs(list(options.items()))

random.shuffle(configs)
results = []

for config in configs[:20]:
    model = create_model(**dict(config))
    hist = model.fit(x, np.expand_dims(y, -1), validation_split=0.15, verbose=0, nb_epoch=20)
    print(dict(config))
    max_acc = max(hist.history['val_acc'])
    print('Validation set accuracy:', max_acc)
    print('Reached at epoch:', hist.history['val_acc'].index(max_acc))
    print()
    results.append((dict(config), max_acc, hist.history['val_acc'].index(max_acc)))

print('done')

{'n_layers': 2, 'layer_size': 128, 'activation': 'relu', 'dropout': 0}
Validation set accuracy: 0.791044777009
Reached at epoch: 16

{'n_layers': 3, 'layer_size': 128, 'activation': 'relu', 'dropout': 0.1}
Validation set accuracy: 0.805970149254
Reached at epoch: 5

{'n_layers': 2, 'layer_size': 512, 'activation': 'sigmoid', 'dropout': 0.1}
Validation set accuracy: 0.828358208066
Reached at epoch: 13

{'n_layers': 1, 'layer_size': 128, 'activation': 'relu', 'dropout': 0}
Validation set accuracy: 0.798507463576
Reached at epoch: 15

{'n_layers': 3, 'layer_size': 256, 'activation': 'relu', 'dropout': 0}
Validation set accuracy: 0.805970149254
Reached at epoch: 0

{'n_layers': 3, 'layer_size': 256, 'activation': 'sigmoid', 'dropout': 0}
Validation set accuracy: 0.813432834931
Reached at epoch: 5

{'n_layers': 1, 'layer_size': 128, 'activation': 'sigmoid', 'dropout': 0.1}
Validation set accuracy: 0.805970149254
Reached at epoch: 1

{'n_layers': 3, 'layer_size': 512, 'activation': 'sigmoid'

In [10]:
# best = max(results, key=lambda x: x[1])
best = ({'activation': 'sigmoid', 'dropout': 0, 'layer_size': 512, 'n_layers': 3},
 0.83582089463276654,
 19)
best

({'activation': 'sigmoid', 'dropout': 0, 'layer_size': 512, 'n_layers': 3},
 0.8358208946327665,
 19)

In [11]:
final_model = create_model(**best[0])
final_model.fit(x, y, nb_epoch=best[2], verbose=0)
test_people = list(DictReader(open('../input/test.csv')))
x_ = np.stack([vectorize(p) for p in test_people])
predictions = final_model.predict_classes(x_)

 32/418 [=>............................] - ETA: 2s

In [15]:
ids = [p['PassengerId'] for p in test_people]
f = open('output.csv', 'w')
writer = csv.writer(f)
writer.writerow(['PassengerId', 'Survived'])
for id, surv in zip(ids, predictions):
    writer.writerow([id, surv])
f.close()

In [16]:
print(open('output.csv').read())

PassengerId,Survived
892,0
893,0
894,0
895,0
896,1
897,0
898,1
899,0
900,1
901,0
902,0
903,0
904,1
905,0
906,1
907,1
908,0
909,0
910,0
911,1
912,0
913,1
914,1
915,0
916,1
917,0
918,1
919,0
920,0
921,0
922,0
923,0
924,0
925,0
926,0
927,0
928,0
929,1
930,0
931,0
932,0
933,0
934,0
935,1
936,1
937,0
938,0
939,0
940,1
941,0
942,0
943,0
944,0
945,1
946,0
947,0
948,0
949,0
950,0
951,1
952,0
953,0
954,0
955,1
956,1
957,1
958,1
959,0
960,0
961,0
962,1
963,0
964,0
965,1
966,1
967,0
968,0
969,1
970,0
971,1
972,1
973,0
974,0
975,0
976,0
977,0
978,1
979,1
980,1
981,1
982,1
983,0
984,1
985,0
986,0
987,0
988,1
989,0
990,1
991,0
992,1
993,0
994,0
995,0
996,1
997,0
998,0
999,0
1000,0
1001,0
1002,0
1003,1
1004,1
1005,1
1006,1
1007,0
1008,0
1009,1
1010,0
1011,1
1012,1
1013,0
1014,1
1015,0
1016,0
1017,1
1018,0
1019,1
1020,0
1021,0
1022,0
1023,0
1024,0
1025,0
1026,0
1027,0
1028,0
1029,0
1030,1
1031,0
1032,0
1033,1
1034,0
1035,0
1036,0
1037,0
1038,1
1039,0
1040,0
1041,0
1042,1
1043,0
1044,0
1045,0
1046,0
10